In [51]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd().parent.parent.parent
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

# ✅ Load events with UTF-8-sig to remove BOM
events = pd.read_csv(DATA_RAW / "events.csv", sep=";", encoding="utf-8-sig")
events.columns = events.columns.str.strip()  # Remove extra spaces

# ✅ Load prices
prices = pd.read_csv(DATA_PROCESSED / "prices_with_returns.csv")

# ✅ Normalize dates FIRST (before merge)
events["date"] = pd.to_datetime(events["date"], dayfirst=True, errors="coerce")
prices["date"] = pd.to_datetime(prices["date"], errors="coerce")

# ✅ Normalize ticker
events["ticker"] = events["ticker"].astype(str).str.upper()
prices["ticker"] = prices["ticker"].astype(str).str.upper()

print("Events columns:", events.columns.tolist())
print("Prices columns:", prices.columns.tolist())

# ✅ Sort before merge
events_sorted = events.sort_values(["ticker", "date"]).reset_index(drop=True)
prices_sorted = prices.sort_values(["ticker", "date"]).reset_index(drop=True)

merged_list = []

for ticker in events_sorted["ticker"].unique():
    e = events_sorted[events_sorted["ticker"] == ticker].copy()
    p = prices_sorted[prices_sorted["ticker"] == ticker].copy()

    if p.empty:
        print(f"[WARN] no price data for ticker {ticker}")
        e["trading_date"] = pd.NaT
        e["adj_close"] = pd.NA
        e["return"] = pd.NA
        e["market_return"] = pd.NA
        merged_list.append(e)
        continue

    # ✅ Rename columns for merge
    e_renamed = e.rename(columns={"date": "event_date"})
    p_renamed = p.rename(columns={"date": "trading_date"})

    # ✅ Ensure dates are datetime (THIS IS KEY!)
    e_renamed["event_date"] = pd.to_datetime(e_renamed["event_date"])
    p_renamed["trading_date"] = pd.to_datetime(p_renamed["trading_date"])

    # ✅ Sort before merge_asof
    e_renamed = e_renamed.sort_values("event_date").reset_index(drop=True)
    p_renamed = p_renamed.sort_values("trading_date").reset_index(drop=True)

    # ✅ Merge
    tmp = pd.merge_asof(
        e_renamed,
        p_renamed[["trading_date", "adj_close", "return", "market_return"]],
        left_on="event_date",
        right_on="trading_date",
        direction="backward",
    )

    merged_list.append(tmp)

merged = pd.concat(merged_list, ignore_index=True)

print(f"\n✅ Merged {len(merged)} rows")
print("Columns:", merged.columns.tolist())

# ✅ Keep only needed columns
cols_to_keep = [
    "event_id", "event_date", "trading_date", "ticker", 
    "is_rockstar", "event_type", "sentiment", 
    "impact_expectation_manual", "adj_close", "return", "market_return"
]
merged = merged[[col for col in cols_to_keep if col in merged.columns]]

print(f"\nFinal columns: {merged.columns.tolist()}")
print(merged.head(10))

# ✅ Save with semicolon separator
out_path = DATA_PROCESSED / "events_with_returns.csv"
merged.to_csv(out_path, sep=";", index=False)
print(f"\n✅ Saved: {out_path}")

Events columns: ['event_id', 'date', 'publisher', 'ticker', 'studio', 'is_rockstar', 'game', 'franchise', 'event_type', 'sentiment', 'impact_expectation_manual', 'source_url', 'notes']
Prices columns: ['date', 'ticker', 'adj_close', 'return', 'market_return']

✅ Merged 72 rows
Columns: ['event_id', 'event_date', 'publisher', 'ticker', 'studio', 'is_rockstar', 'game', 'franchise', 'event_type', 'sentiment', 'impact_expectation_manual', 'source_url', 'notes', 'trading_date', 'adj_close', 'return', 'market_return']

Final columns: ['event_id', 'event_date', 'trading_date', 'ticker', 'is_rockstar', 'event_type', 'sentiment', 'impact_expectation_manual', 'adj_close', 'return', 'market_return']
                         event_id event_date trading_date ticker is_rockstar  \
0      ATVI_2019_CODMOBILE_LAUNCH 2019-10-01   2019-10-01   ATVI           0   
1         ATVI_2019_CODMW_RELEASE 2019-10-25   2019-10-25   ATVI           0   
2    ATVI_2020_WARCRAFT3_REFORGED 2020-01-28   2020-01-28   AT

/tmp/ipykernel_680/3668211594.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events["date"] = pd.to_datetime(events["date"], dayfirst=True, errors="coerce")


In [41]:
from pathlib import Path

# This should match your project root
BASE_DIR = Path.cwd().parent.parent.parent

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

print("BASE_DIR:", BASE_DIR)
print("DATA_RAW exists:", DATA_RAW.exists())
print("DATA_PROCESSED exists:", DATA_PROCESSED.exists())


BASE_DIR: /files/capstone_project/game-market-event-analyzer
DATA_RAW exists: True
DATA_PROCESSED exists: True


In [42]:
import pandas as pd

events_path = DATA_RAW / "events.csv"

# Use the correct encoding (try latin-1 first - most common for European data)
events = pd.read_csv(events_path, sep=";", encoding="latin-1")

print("Columns:", events.columns.tolist())
print(f"✅ Loaded {len(events)} events")
events.head()

Columns: ['ï»¿event_id', 'date', 'publisher', 'ticker', 'studio', 'is_rockstar', 'game', 'franchise', 'event_type', 'sentiment', 'impact_expectation_manual', 'source_url', 'notes']
✅ Loaded 72 events


,ï»¿event_id,date,publisher,ticker,studio,is_rockstar,game,franchise,event_type,sentiment,impact_expectation_manual,source_url,notes
0,TTWO_2011_GTA5_ANNOUNCEMENT,25.10.11,Take-Two,TTWO,Rockstar Games,1,GTA V,GTA,Major_event,Positive,Medium,https://www.rockstargames.com/newswire/article...,Rockstar announces Grand Theft Auto V with fir...
1,GTAV_2011_TRAILER1,02.11.11,Take-Two,TTWO,Rockstar,1,GTA V,GTA,Trailer,positive,medium,https://www.rockstargames.com,GTA V Trailer #1
2,GTAV_2012_SCREENSHOTS,12.07.12,Take-Two,TTWO,Rockstar,1,GTA V,GTA,Screenshots,neutral,low,https://www.rockstargames.com,First screenshot batch
3,GTAV_2012_TRAILER2,14.11.12,Take-Two,TTWO,Rockstar,1,GTA V,GTA,Trailer,positive,medium,https://www.rockstargames.com,GTA V Trailer #2
4,GTAV_2013_DELAY,31.01.13,Take-Two,TTWO,Rockstar,1,GTA V,GTA,Delay,negative,medium,https://www.rockstargames.com,Delay to September 2013


In [43]:
# Normalize date and ticker
events["date"] = pd.to_datetime(events["date"], dayfirst=True, errors="coerce")
events["ticker"] = events["ticker"].astype(str).str.upper()

# Fix Ubisoft ticker: map empty/UBI.PA to UBSFY
events.loc[events["publisher"] == "Ubisoft", "ticker"] = "UBSFY"
events["ticker"] = events["ticker"].str.replace("UBI.PA", "UBSFY", regex=False)

events = pd.read_csv(DATA_RAW / "events.csv", sep=";", encoding="latin-1")

# ✅ Remove BOM from column names
events.columns = events.columns.str.replace("ï»¿", "", regex=False)

print("Columns:", events.columns.tolist())
print(events[["event_id", "date", "ticker", "publisher"]].head(15))
print("Unique tickers in events:", events["ticker"].unique())


Columns: ['event_id', 'date', 'publisher', 'ticker', 'studio', 'is_rockstar', 'game', 'franchise', 'event_type', 'sentiment', 'impact_expectation_manual', 'source_url', 'notes']
                         event_id      date ticker publisher
0     TTWO_2011_GTA5_ANNOUNCEMENT  25.10.11   TTWO  Take-Two
1              GTAV_2011_TRAILER1  02.11.11   TTWO  Take-Two
2           GTAV_2012_SCREENSHOTS  12.07.12   TTWO  Take-Two
3              GTAV_2012_TRAILER2  14.11.12   TTWO  Take-Two
4                 GTAV_2013_DELAY  31.01.13   TTWO  Take-Two
5          TTWO_2013_GTA5_RELEASE  17.09.13   TTWO  Take-Two
6      TTWO_2013_GTAONLINE_LAUNCH  01.10.13   TTWO  Take-Two
7      TTWO_2014_GTAONLINE_HEISTS  09.12.14   TTWO  Take-Two
8       TTWO_2015_GTA5_PC_RELEASE  14.04.15   TTWO  Take-Two
9     TTWO_2016_RDR2_ANNOUNCEMENT  18.10.16   TTWO  Take-Two
10             RDR2_2016_TRAILER1  20.10.16   TTWO  Take-Two
11                RDR2_2017_DELAY  01.02.17   TTWO  Take-Two
12  NTDOY_2017_ZELDA_BOTW_REL

/tmp/ipykernel_680/1361933251.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events["date"] = pd.to_datetime(events["date"], dayfirst=True, errors="coerce")


In [44]:
import pandas as pd

# Load prices to see if UBSFY exists
prices_path = DATA_PROCESSED / "prices_with_returns.csv"
prices = pd.read_csv(prices_path)

# ✅ CONVERT DATE TO DATETIME
prices["date"] = pd.to_datetime(prices["date"])
prices["ticker"] = prices["ticker"].astype(str).str.upper()

print("Unique tickers in prices:")
print(prices["ticker"].unique())
print(f"\nUBSFY rows: {len(prices[prices['ticker'] == 'UBSFY'])}")

# Check date range for UBSFY
ubsfy_prices = prices[prices['ticker'] == 'UBSFY']
print(f"\nUBSFY date range: {ubsfy_prices['date'].min()} to {ubsfy_prices['date'].max()}")

# Check Ubisoft event dates
ubsfy_events = events[events['ticker'] == 'UBSFY']
print(f"\nUbisoft event dates:")
print(ubsfy_events[['event_id', 'date', 'ticker']])

Unique tickers in prices:
['ATVI' 'EA' 'NTDOY' 'SP500' 'TTWO' 'UBSFY']

UBSFY rows: 3993

UBSFY date range: 2010-01-04 00:00:00 to 2025-11-14 00:00:00

Ubisoft event dates:
                                 event_id      date ticker
24  UBI_2019_GHOSTRECON_BREAKPOINT_LAUNCH  04.10.19  UBSFY
25            UBI_2019_UBI_PROFIT_WARNING  24.10.19  UBSFY
32              UBI_2020_SKULLBONES_DELAY  12.07.20  UBSFY
34           UBI_2020_AC_VALHALLA_RELEASE  10.11.20  UBSFY
45        UBI_2023_AC_MIRAGE_ANNOUNCEMENT  10.09.22  UBSFY
47         UBI_2022_SKULL_AND_BONES_DELAY  28.09.22  UBSFY
54                        UBI_EARN_FY2023  16.05.23  UBSFY
58       UBI_2023_ASSASSINSMIRAGE_RELEASE  05.10.23  UBSFY
59                       UBI_EARN_H1_2024  07.11.23  UBSFY
66                        UBI_EARN_FY2024  16.05.24  UBSFY


In [ ]:
print(f"✅ Merged {len(merged)} rows")
print("Columns:", merged.columns.tolist())

# Clean up duplicate columns from merge
if "ticker_x" in merged.columns and "ticker_y" in merged.columns:
    merged = merged.drop("ticker_y", axis=1)
    merged = merged.rename(columns={"ticker_x": "ticker"})

# Keep only the columns you need
cols_to_keep = ["event_id", "event_date", "trading_date", "ticker", "is_rockstar", "event_type", "sentiment", "impact_expectation_manual", "adj_close", "return", "market_return"]
merged = merged[[col for col in cols_to_keep if col in merged.columns]]

print(merged.head(10))

✅ Merged 72 rows
Columns: ['ï»¿event_id', 'event_date', 'publisher', 'ticker_x', 'studio', 'is_rockstar', 'game', 'franchise', 'event_type', 'sentiment', 'impact_expectation_manual', 'source_url', 'notes', 'trading_date', 'ticker_y', 'adj_close', 'return', 'market_return']
  event_date trading_date ticker is_rockstar    event_type sentiment  \
0 2019-10-01   2019-10-01   ATVI           0  Game_release  Positive   
1 2019-10-25   2019-10-25   ATVI           0  Game_release  Positive   
2 2020-01-28   2020-01-28   ATVI           0    Bad_review  Negative   
3 2020-03-10   2020-03-10   ATVI           0  Game_release  Positive   
4 2021-07-20   2021-07-20   ATVI           0       Lawsuit  negative   
5 2021-09-23   2021-09-23   ATVI           0  Game_release   Neutral   
6 2022-01-18   2022-01-18   ATVI           0   Major_event  Positive   
7 2023-07-19   2023-07-19   ATVI           0      Earnings       NaN   
8 2023-10-04   2023-10-04   ATVI           0      Earnings       NaN   
9 2017

In [46]:
print("Rows:", len(merged))
print("NaN in adj_close:", merged["adj_close"].isna().sum())
print("NaN in return:", merged["return"].isna().sum())
print("NaN in market_return:", merged["market_return"].isna().sum())

merged[["event_id", "event_date", "trading_date", "ticker", "is_rockstar", "event_type", "sentiment", "impact_expectation_manual", "adj_close", "return", "market_return"]].head(10)

Rows: 72
NaN in adj_close: 0
NaN in return: 0
NaN in market_return: 0


,event_id,event_date,trading_date,ticker,is_rockstar,event_type,sentiment,impact_expectation_manual,adj_close,return,market_return
0,ATVI_2019_CODMOBILE_LAUNCH,2019-10-01,2019-10-01,ATVI,0,Game_release,Positive,Medium,94.157463,-0.010938,-0.012258
1,ATVI_2019_CODMW_RELEASE,2019-10-25,2019-10-25,ATVI,0,Game_release,Positive,High,93.729248,0.003438,0.004073
2,ATVI_2020_WARCRAFT3_REFORGED,2020-01-28,2020-01-28,ATVI,0,Bad_review,Negative,Medium,108.901489,0.012120,0.010054
3,ATVI_2020_WARZONE_LAUNCH,2020-03-10,2020-03-10,ATVI,0,Game_release,Positive,High,100.609787,0.024274,0.049396
4,ATVI_2021_LAWSUIT,2021-07-20,2021-07-20,ATVI,0,Lawsuit,negative,high,137.807159,-0.001204,0.015163
5,ATVI_2021_DIABLO2_RESURRECTED,2021-09-23,2021-09-23,ATVI,0,Game_release,Neutral,Medium,125.080391,0.011788,0.012135
6,ATVI_2022_MSFT_ACQUISITION_ANN,2022-01-18,2022-01-18,ATVI,0,Major_event,Positive,High,131.144638,0.026602,-0.018388
7,ATVI_EARN_2023Q2,2023-07-19,2023-07-19,ATVI,0,Earnings,NaN,NaN,135.207016,-0.014618,0.002358
8,ATVI_EARN_2023Q3,2023-10-04,2023-10-04,ATVI,0,Earnings,NaN,NaN,119.467735,0.009787,0.008110
9,EA_2017_SWBF2_CONTROVERSY,2017-11-17,2017-11-17,EA,0,Bad_review,Negative,High,105.904030,-0.024910,-0.002626


In [47]:
merged[merged["event_id"].str.contains("GTA6", case=False, na=False)][
    ["event_id", "event_date", "trading_date", "ticker", "adj_close", "return", "market_return"]
]

,event_id,event_date,trading_date,ticker,adj_close,return,market_return
49,TTWO_2022_GTA6_DEV-ANNOUNCEMENT,2022-02-04,2022-02-04,TTWO,175.000000,0.073488,0.005157
52,GTA6_2022_LEAK,2022-09-18,2022-09-16,TTWO,124.190002,-0.033014,-0.007182
55,TTWO_2023_GTA6_TRAILER1,2023-12-05,2023-12-05,TTWO,156.759995,-0.005077,-0.000569
57,TTWO_2024_STRONG_RESULTS_GTA6_HYPE,2024-05-16,2024-05-16,TTWO,146.080002,-0.013506,-0.002082
58,GTA6_2024_KEYART,2024-05-16,2024-05-16,TTWO,146.080002,-0.013506,-0.002082
59,TTWO_2025_GTA6_DELAY1,2025-05-02,2025-05-02,TTWO,219.500000,-0.066633,0.014727
60,TTWO_2025_GTA6_TRAILER2,2025-05-06,2025-05-06,TTWO,231.839996,0.028937,-0.007693
61,TTWO_2025_GTA6_DELAY2,2025-11-06,2025-11-06,TTWO,252.399994,-0.009264,-0.011178
